In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units
from pathlib import Path
import os

import snapshot_obj
import dataset_compute

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)

# Mass distribution of satellites of M31 and MW

Count the accumulation of satellites with $v_\mathrm{max}$ from the centres of potential of the MW and M31 galaxies **separately**.

Create a dictionary of the datasets from each simulation. 

In [ ]:
snap_id = 127
sim_ids = ["V1_MR_fix", "V1_MR_curvaton_p082_fix", "V1_LR_fix", "V1_LR_curvaton_p082_fix", "V1_LR_curvaton_p084_fix"]
names = ["plain-LCDM", "spec-p082", "plain-LCDM-LR", "spec-p082-LR", "spec-p084-LR"]

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0), (1,0), (1,0), (1,0)]
mw = [(2,0), (1,1), (2,0), (1,1), (1,0)]

data = {"By_r" : {}, "By_GN" : {}}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    for entry in data.values():
        entry[name] = {"snapshot": snapshot_obj.Snapshot(sim_id, snap_id, name=name),
                       "M31_identifier": m31_ns,
                       "MW_identifier": mw_ns}

In [ ]:
z = data["By_r"]["plain-LCDM"]["snapshot"].get_attribute("Redshift", "Header")

## Get data for plotting

Choose how to distinguish between satellites and isolated:

In [ ]:
distinction = "By_r"

Read the datasets from each simulation, convert the units and split into satellites/isolated and luminous/dark:

In [ ]:
for key, sim_data in data[distinction].items():    
    # Get data:
    snap = sim_data["snapshot"]
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    
    # Add M31 and MW to the dictionary:
    idx_m31 = snap.index_of_halo(sim_data["M31_identifier"][0], sim_data["M31_identifier"][1])
    sim_data["M31"] = {"Vmax": vmax[idx_m31]}
    idx_mw = snap.index_of_halo(sim_data["MW_identifier"][0], sim_data["MW_identifier"][1])
    sim_data["MW"] = {"Vmax": vmax[idx_mw]}
    
    # Compute masking arrays:
    mask_lum, mask_dark = dataset_compute.split_luminous(snap)
    mask_nonzero_vmax = dataset_compute.prune_vmax(snap)
    if distinction == "By_r":
        mask_sat, mask_isol = dataset_compute.split_satellites_by_distance(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    elif distinction == "By_GN":
        mask_sat, mask_isol = dataset_compute.split_satellites_by_group_number(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
        
    # Sort by vmax and add a dummy point with very small vmax 
    # (to continue the curves to the y-axis):
    sort_idx = np.argsort(vmax)
    vmax = np.concatenate([[0.01], vmax[sort_idx]])
    sim_data["Vmax"] = vmax
    mask_sat = [np.concatenate([[True], mask_sat[0][sort_idx]]),
                np.concatenate([[True], mask_sat[1][sort_idx]])]
    mask_isol = np.concatenate([[True], mask_isol[sort_idx]])
    mask_lum = np.concatenate([[True], mask_lum[sort_idx]])
    mask_dark = np.concatenate([[True], mask_dark[sort_idx]])
    mask_nonzero_vmax = np.concatenate([[True], mask_nonzero_vmax[sort_idx]])

    # Add these selections to the data dict:
    sim_data["Selections"] = {
        "M31" : {
            "Luminous" : np.logical_and.reduce([mask_sat[0], mask_lum, mask_nonzero_vmax]),
            "Dark" : np.logical_and.reduce([mask_sat[0], mask_dark, mask_nonzero_vmax])
        },
        "MW" : {
            "Luminous" : np.logical_and.reduce([mask_sat[1], mask_lum, mask_nonzero_vmax]),
            "Dark" : np.logical_and.reduce([mask_sat[1], mask_dark, mask_nonzero_vmax])
        },
        "Isolated" : {
            "Luminous" : np.logical_and.reduce([mask_isol, mask_lum, mask_nonzero_vmax]),
            "Dark" : np.logical_and.reduce([mask_isol, mask_dark, mask_nonzero_vmax])
        }
    }
    
    # In case M31 and MW are identified as the same halo, remove intersection:
    sim_data["Selections"]["MW"] = {
        "Luminous" : np.logical_and(
            sim_data["Selections"]["MW"]["Luminous"],
            np.logical_not(sim_data["Selections"]["M31"]["Luminous"])
        ),
        "Dark" : np.logical_and(
            sim_data["Selections"]["MW"]["Dark"],
            np.logical_not(sim_data["Selections"]["M31"]["Dark"])
        )
    }

## Plot all satellites and isolated galaxies

In [ ]:
# NOTE THE HIGH VALUE OF y_up (and low value of x_down): 
# IN THE MR CASE THE HALO NUMBER CONTINUES TO GROW DOWN TO v_max ~ 5 km/s

# Set some parameters:
x_down = 1; x_up = 110
y_down = 1; y_up = 5000

# Set colors:
color = ["black", "red", "blue", "green"]

In [ ]:
# Set some parameters:
x_down = 7; x_up = 110
y_down = 1; y_up = 1000

# Set colors:
color = ["black", "red", "blue", "green"]

In [ ]:
# Construct saving location:
filename = "mass_distribution_sat_and_isol"
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures", "MediumResolution")    
filename = os.path.join(path, filename)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(8,4), sharey="row")
plt.subplots_adjust(wspace=0)

# Set axis:
axes[0].set_ylim(y_down, y_up)
axes[0].set_ylabel('$N(>v_{\mathrm{max}})$')
for ax in axes:
    ax.set_xlim(x_down, x_up)
    ax.set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$')
    ax.set_xscale('log')
    ax.set_yscale('log')

# Add redshift:
# z_x = 10 ** (np.log10(x_up) - 9/10 * (np.log10(x_up)-np.log10(x_down)))
# z_y = 10 ** (np.log10(y_up) - 1/10 * (np.log10(y_up)-np.log10(y_down)))
# axes[0].text(z_x, z_y, "z = {:.2f}".format(z))
    
axes[0].set_title('Satellite galaxies')
axes[1].set_title('Isolated galaxies')

# Add scatter plots:
color = [['black', 'gray', 'lightgray'], ['red', 'pink', 'lightpink'],
         ['gray'], ['pink'],
         ['lightblue']]

# First, plot LR on the background:
for i, (name, entry) in enumerate(data["By_r"].items()):
    
    if name.split("-")[-1] != "LR":
        continue
               
    vmax = entry["Vmax"]
        
    # ISOLATED GALAXIES
    # -----------------
    
    mask_lum = entry["Selections"]["Isolated"]["Luminous"]
    mask_dark = entry["Selections"]["Isolated"]["Dark"]
    mask_all = np.logical_or(mask_lum, mask_dark)
    
    # Plot luminous isolated:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[1].plot(vmax[mask_lum], cnt_lum,
                 c=color[i][0], linestyle='dashed')
    
    # Plot all isolated:
    cnt_all = np.arange(1, np.sum(mask_all) + 1)[::-1] 
    axes[1].plot(vmax[mask_all], cnt_all,
                 c=color[i][0], linestyle='solid')


    # SATELLITES
    # ----------
        
    if name.split("-")[1] == "p084":
        continue

    mask_lum = np.logical_or(entry["Selections"]["M31"]["Luminous"],
                             entry["Selections"]["MW"]["Luminous"])
    mask_dark = np.logical_or(entry["Selections"]["M31"]["Dark"],
                              entry["Selections"]["MW"]["Dark"])
    mask_all = np.logical_or(mask_lum, mask_dark)
    
    # Plot luminous satellites:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[0].plot(vmax[mask_lum], cnt_lum,
                 c=color[i][0], linestyle='dashed')
    
    # Plot all satellites:
    cnt = np.arange(1, np.sum(mask_all) + 1)[::-1] 
    axes[0].plot(vmax[mask_all], cnt,
                 c=color[i][0], linestyle='solid')

    
# Then, plot MR:
for i, (name, entry) in enumerate(data["By_r"].items()):
    
    if name.split("-")[-1] == "LR":
        continue
        
    vmax = entry["Vmax"]
    
    # SATELLITES
    # ----------
    
    mask_lum = np.logical_or(entry["Selections"]["M31"]["Luminous"],
                             entry["Selections"]["MW"]["Luminous"])
    mask_dark = np.logical_or(entry["Selections"]["M31"]["Dark"],
                              entry["Selections"]["MW"]["Dark"])
    mask_all = np.logical_or(mask_lum, mask_dark)
    
    # Plot all luminous satellites:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[0].plot(vmax[mask_lum], cnt_lum,
                 c=color[i][0], linestyle='dashed')
    
    # Plot all satellites, luminous and dark:
    cnt_all = np.arange(1, np.sum(mask_all) + 1)[::-1]
    axes[0].plot(vmax[mask_all], cnt_all,
                 c=color[i][0], linestyle='solid')
  
    # Fill area of dark satellites:
    axes[0].fill(
        np.append(vmax[mask_lum], vmax[mask_all][::-1]),
        np.append(cnt_lum, cnt_all[::-1]),
        color=color[i][2],
        alpha=0.3
    )
        
    # Plot M31 luminous satellites:
#     mask_m31_lum = entry["Selections"]["M31"]["Luminous"]
#     cnt_m31_lum = np.arange(1, np.sum(mask_m31_lum) + 1)[::-1] 
#     axes[0].plot(vmax[mask_m31_lum], cnt_m31_lum, 
#                  c=color[i][0], linestyle='dotted', label='{} luminous'.format(name))
    

    # ISOLATED GALAXIES
    # -----------------
    
    mask_lum = entry["Selections"]["Isolated"]["Luminous"]
    mask_dark = entry["Selections"]["Isolated"]["Dark"]
    mask_all = np.logical_or(mask_lum, mask_dark)
    
    # Plot luminous isolated:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[1].plot(vmax[mask_lum], cnt_lum,
                 c=color[i][0], linestyle='dashed')
    
    
    # Plot all isolated:
    cnt_all = np.arange(1, np.sum(mask_all) + 1)[::-1]
    axes[1].plot(vmax[mask_all], cnt_all,
                 c=color[i][0], linestyle='solid')
  
    # Fill area of dark isolated:
    axes[1].fill(
        np.append(vmax[mask_lum], vmax[mask_all][::-1]),
        np.append(cnt_lum, cnt_all[::-1]),
        color=color[i][2],
        alpha=0.3
    )

            
# Make dummy plots for the legend:   
l = []
for c in color:
    l_dum, = axes[1].plot([], [], c=c[0], linestyle="solid")
    l.append(l_dum)
sim_legend = axes[1].legend(l, list(data["By_r"].keys()), loc="upper right")
axes[1].add_artist(sim_legend)


axes[0].plot([], [], c=color[0][0], linestyle="solid", label="All")
axes[0].plot([], [], c=color[0][0], linestyle="dashed", label="Luminous")
axes[0].legend(loc="upper right")
    
plt.tight_layout()
plt.savefig(filename, dpi=200)

## Plot Andromeda and Milky Way separately

Add the other satellite/isolated distinction to the data dictionary:

In [ ]:
distinction = "By_GN"

In [ ]:
print(list(data.keys()))

Read the datasets from each simulation, convert the units and split into satellites/isolated and luminous/dark:

In [ ]:
for key, sim_data in data[distinction].items():    
    # Get data:
    snap = sim_data["snapshot"]
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    
    # Add M31 and MW to the dictionary:
    idx_m31 = snap.index_of_halo(sim_data["M31_identifier"][0], sim_data["M31_identifier"][1])
    sim_data["M31"] = {"Vmax": vmax[idx_m31]}
    idx_mw = snap.index_of_halo(sim_data["MW_identifier"][0], sim_data["MW_identifier"][1])
    sim_data["MW"] = {"Vmax": vmax[idx_mw]}
    
    # Compute masking arrays:
    mask_lum, mask_dark = dataset_compute.split_luminous(snap)
    mask_nonzero_vmax = dataset_compute.prune_vmax(snap)
    if distinction == "By_r":
        mask_sat, mask_isol = dataset_compute.split_satellites_by_distance(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    elif distinction == "By_GN":
        mask_sat, mask_isol = dataset_compute.split_satellites_by_group_number(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
        
    # Sort by vmax and add a dummy point with very small vmax 
    # (to continue the curves to the y-axis):
    sort_idx = np.argsort(vmax)
    vmax = np.concatenate([[0.01], vmax[sort_idx]])
    sim_data["Vmax"] = vmax
    mask_sat = [np.concatenate([[True], mask_sat[0][sort_idx]]),
                np.concatenate([[True], mask_sat[1][sort_idx]])]
    mask_isol = np.concatenate([[True], mask_isol[sort_idx]])
    mask_lum = np.concatenate([[True], mask_lum[sort_idx]])
    mask_dark = np.concatenate([[True], mask_dark[sort_idx]])
    mask_nonzero_vmax = np.concatenate([[True], mask_nonzero_vmax[sort_idx]])

    # Add these selections to the data dict:
    sim_data["Selections"] = {
        "M31" : {
            "Luminous" : np.logical_and.reduce([mask_sat[0], mask_lum, mask_nonzero_vmax]),
            "Dark" : np.logical_and.reduce([mask_sat[0], mask_dark, mask_nonzero_vmax])
        },
        "MW" : {
            "Luminous" : np.logical_and.reduce([mask_sat[1], mask_lum, mask_nonzero_vmax]),
            "Dark" : np.logical_and.reduce([mask_sat[1], mask_dark, mask_nonzero_vmax])
        },
        "Isolated" : {
            "Luminous" : np.logical_and.reduce([mask_isol, mask_lum, mask_nonzero_vmax]),
            "Dark" : np.logical_and.reduce([mask_isol, mask_dark, mask_nonzero_vmax])
        }
    }
    
    # In case M31 and MW are identified as the same halo, remove intersection:
    sim_data["Selections"]["MW"] = {
        "Luminous" : np.logical_and(
            sim_data["Selections"]["MW"]["Luminous"],
            np.logical_not(sim_data["Selections"]["M31"]["Luminous"])
        ),
        "Dark" : np.logical_and(
            sim_data["Selections"]["MW"]["Dark"],
            np.logical_not(sim_data["Selections"]["M31"]["Dark"])
        )
    }

## Plot

In [ ]:
# NOTE THE HIGH VALUE OF y_up (and low value of x_down): 
# IN THE MR CASE THE HALO NUMBER CONTINUES TO GROW DOWN TO v_max ~ 5 km/s

# Set some parameters:
x_down = 1; x_up = 110
y_down = 1; y_up = 5000

# Set colors:
color = ["black", "red", "blue", "green"]

In [ ]:
# Set some parameters:
x_down = 1; x_up = 110
y_down = 1; y_up = 1000

# Set colors:
color = ["black", "red", "blue", "green"]

In [ ]:
# Construct saving location:
filename = "mass_distribution_sat_and_isol"
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures", "MediumResolution")    
filename = os.path.join(path, filename)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, sharex="col", sharey="row")#, figsize=(8,4))
# plt.subplots_adjust(wspace=0)

# Set axis:

for i in range(np.size(axes, axis=0)):
    axes[i,0].set_yscale("log")
    axes[i,0].set_ylim(y_down, y_up)
    axes[i,0].set_ylabel('$N(>v_{\mathrm{max}})$')

for i in range(np.size(axes, axis=1)):
    axes[0,i].set_xscale("log")
    axes[-1,i].set_xlim(x_down, x_up)
    axes[-1,i].set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$')
    
# axes[0].set_title('Satellite galaxies')
# axes[1].set_title('Isolated galaxies')

color = [['black', 'gray', 'lightgray'], ['red', 'pink', 'lightpink']]

# First, plot distinction by GN:
for i, (name, entry) in enumerate(data["By_GN"].items()):
    
    if name.split("-")[-1] == "LR":
        continue
               
    vmax = entry["Vmax"]
    
    # M31 SATELLITES
    # --------------

    mask_lum = entry["Selections"]["M31"]["Luminous"]
    mask_dark = entry["Selections"]["M31"]["Dark"]
    mask_all = np.logical_or(mask_lum, mask_dark)
    
    # Plot luminous satellites:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[0,0].plot(vmax[mask_lum], cnt_lum,
                   c=color[i][0], linestyle='dashed')
    
    # Plot all satellites:
    cnt = np.arange(1, np.sum(mask_all) + 1)[::-1] 
    axes[0,0].plot(vmax[mask_all], cnt,
                 c=color[i][0], linestyle='solid')
        
    # MW SATELLITES
    # --------------

    mask_lum = entry["Selections"]["MW"]["Luminous"]
    mask_dark = entry["Selections"]["MW"]["Dark"]
    mask_all = np.logical_or(mask_lum, mask_dark)
    
    # Plot luminous satellites:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[1,0].plot(vmax[mask_lum], cnt_lum,
                   c=color[i][0], linestyle='dashed')
    
    # Plot all satellites:
    cnt_all = np.arange(1, np.sum(mask_all) + 1)[::-1] 
    axes[1,0].plot(vmax[mask_all], cnt_all,
                   c=color[i][0], linestyle='solid')
      
    # Fill area of dark satellites:
    axes[1,0].fill(
        np.append(vmax[mask_lum], vmax[mask_all][::-1]),
        np.append(cnt_lum, cnt_all[::-1]),
        color=color[i][2],
        alpha=0.3
    )
    
        
    # ISOLATED GALAXIES
    # -----------------
    
#     mask_lum = entry["Selections"]["Isolated"]["Luminous"]
#     mask_dark = entry["Selections"]["Isolated"]["Dark"]
#     mask_all = np.logical_or(mask_lum, mask_dark)
    
#     # Plot luminous isolated:
#     cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
#     axes[1].plot(vmax[mask_lum], cnt_lum,
#                  c=color[i][0], linestyle='dashed')
    
#     # Plot all isolated:
#     cnt_all = np.arange(1, np.sum(mask_all) + 1)[::-1] 
#     axes[1].plot(vmax[mask_all], cnt_all,
#                  c=color[i][0], linestyle='solid')


    
# # Then, plot MR:
# for i, (name, entry) in enumerate(data.items()):
    
#     if name.split("-")[-1] == "LR":
#         continue
        
#     vmax = entry["Vmax"]
    
#     # SATELLITES
#     # ----------
    
#     mask_lum = np.logical_or(entry["Selections"]["M31"]["Luminous"],
#                              entry["Selections"]["MW"]["Luminous"])
#     mask_dark = np.logical_or(entry["Selections"]["M31"]["Dark"],
#                               entry["Selections"]["MW"]["Dark"])
#     mask_all = np.logical_or(mask_lum, mask_dark)
    
#     # Plot all luminous satellites:
#     cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
#     axes[0].plot(vmax[mask_lum], cnt_lum,
#                  c=color[i][0], linestyle='dashed')
    
#     # Plot all satellites, luminous and dark:
#     cnt_all = np.arange(1, np.sum(mask_all) + 1)[::-1]
#     axes[0].plot(vmax[mask_all], cnt_all,
#                  c=color[i][0], linestyle='solid')
  
#     # Fill area of dark satellites:
#     axes[0].fill(
#         np.append(vmax[mask_lum], vmax[mask_all][::-1]),
#         np.append(cnt_lum, cnt_all[::-1]),
#         color=color[i][2],
#         alpha=0.3
#     )
        
#     # Plot M31 luminous satellites:
# #     mask_m31_lum = entry["Selections"]["M31"]["Luminous"]
# #     cnt_m31_lum = np.arange(1, np.sum(mask_m31_lum) + 1)[::-1] 
# #     axes[0].plot(vmax[mask_m31_lum], cnt_m31_lum, 
# #                  c=color[i][0], linestyle='dotted', label='{} luminous'.format(name))
    

#     # ISOLATED GALAXIES
#     # -----------------
    
#     mask_lum = entry["Selections"]["Isolated"]["Luminous"]
#     mask_dark = entry["Selections"]["Isolated"]["Dark"]
#     mask_all = np.logical_or(mask_lum, mask_dark)
    
#     # Plot luminous isolated:
#     cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
#     axes[1].plot(vmax[mask_lum], cnt_lum,
#                  c=color[i][0], linestyle='dashed')
    
    
#     # Plot all isolated:
#     cnt_all = np.arange(1, np.sum(mask_all) + 1)[::-1]
#     axes[1].plot(vmax[mask_all], cnt_all,
#                  c=color[i][0], linestyle='solid')
  
#     # Fill area of dark isolated:
#     axes[1].fill(
#         np.append(vmax[mask_lum], vmax[mask_all][::-1]),
#         np.append(cnt_lum, cnt_all[::-1]),
#         color=color[i][2],
#         alpha=0.3
#     )

            
# # Make dummy plots for the legend:   
# l = []
# for c in color:
#     l_dum, = axes[1].plot([], [], c=c[0], linestyle="solid")
#     l.append(l_dum)
# sim_legend = axes[1].legend(l, list(data.keys()), loc="upper right")
# axes[1].add_artist(sim_legend)


# axes[0].plot([], [], c=color[0][0], linestyle="solid", label="All")
# axes[0].plot([], [], c=color[0][0], linestyle="dashed", label="Luminous")
# axes[0].legend(loc="upper right")
    
plt.savefig("testi.png", dpi=200)
# plt.tight_layout()
# plt.savefig(filename, dpi=200)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(8,4), sharey="row")
plt.subplots_adjust(wspace=0)

# Set axis:
axes[0].set_ylim(y_down, y_up)
axes[0].set_ylabel('$N(>v_{\mathrm{max}})$')
for ax in axes:
    ax.set_xlim(x_down, x_up)
    ax.set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$')
    ax.set_xscale('log')
    ax.set_yscale('log')

# Add redshift:
# z_x = 10 ** (np.log10(x_up) - 9/10 * (np.log10(x_up)-np.log10(x_down)))
# z_y = 10 ** (np.log10(y_up) - 1/10 * (np.log10(y_up)-np.log10(y_down)))
# axes[0].text(z_x, z_y, "z = {:.2f}".format(z))
    
axes[0].set_title('Satellite galaxies')
axes[1].set_title('Isolated galaxies')

# Add scatter plots:
color = [['black', 'gray', 'lightgray'], ['red', 'pink', 'lightpink'],
         ['gray'], ['pink'],
         ['lightblue']]

# First, plot LR on the background:
for i, (name, entry) in enumerate(data.items()):
    
    if name.split("-")[-1] != "LR":
        continue
               
    vmax = entry["Vmax"]
        
    # ISOLATED GALAXIES
    # -----------------
    
    mask_lum = entry["Selections"]["Isolated"]["Luminous"]
    mask_dark = entry["Selections"]["Isolated"]["Dark"]
    mask_all = np.logical_or(mask_lum, mask_dark)
    
    # Plot luminous isolated:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[1].plot(vmax[mask_lum], cnt_lum,
                 c=color[i][0], linestyle='dashed')
    
    # Plot all isolated:
    cnt_all = np.arange(1, np.sum(mask_all) + 1)[::-1] 
    axes[1].plot(vmax[mask_all], cnt_all,
                 c=color[i][0], linestyle='solid')


    # SATELLITES
    # ----------
        
    if name.split("-")[1] == "p084":
        continue

    mask_lum = np.logical_or(entry["Selections"]["M31"]["Luminous"],
                             entry["Selections"]["MW"]["Luminous"])
    mask_dark = np.logical_or(entry["Selections"]["M31"]["Dark"],
                              entry["Selections"]["MW"]["Dark"])
    mask_all = np.logical_or(mask_lum, mask_dark)
    
    # Plot luminous satellites:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[0].plot(vmax[mask_lum], cnt_lum,
                 c=color[i][0], linestyle='dashed')
    
    # Plot all satellites:
    cnt = np.arange(1, np.sum(mask_all) + 1)[::-1] 
    axes[0].plot(vmax[mask_all], cnt,
                 c=color[i][0], linestyle='solid')

    
# Then, plot MR:
for i, (name, entry) in enumerate(data.items()):
    
    if name.split("-")[-1] == "LR":
        continue
        
    vmax = entry["Vmax"]
    
    # SATELLITES
    # ----------
    
    mask_lum = np.logical_or(entry["Selections"]["M31"]["Luminous"],
                             entry["Selections"]["MW"]["Luminous"])
    mask_dark = np.logical_or(entry["Selections"]["M31"]["Dark"],
                              entry["Selections"]["MW"]["Dark"])
    mask_all = np.logical_or(mask_lum, mask_dark)
    
    # Plot all luminous satellites:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[0].plot(vmax[mask_lum], cnt_lum,
                 c=color[i][0], linestyle='dashed')
    
    # Plot all satellites, luminous and dark:
    cnt_all = np.arange(1, np.sum(mask_all) + 1)[::-1]
    axes[0].plot(vmax[mask_all], cnt_all,
                 c=color[i][0], linestyle='solid')
  
    # Fill area of dark satellites:
    axes[0].fill(
        np.append(vmax[mask_lum], vmax[mask_all][::-1]),
        np.append(cnt_lum, cnt_all[::-1]),
        color=color[i][2],
        alpha=0.3
    )
        
    # Plot M31 luminous satellites:
#     mask_m31_lum = entry["Selections"]["M31"]["Luminous"]
#     cnt_m31_lum = np.arange(1, np.sum(mask_m31_lum) + 1)[::-1] 
#     axes[0].plot(vmax[mask_m31_lum], cnt_m31_lum, 
#                  c=color[i][0], linestyle='dotted', label='{} luminous'.format(name))
    

    # ISOLATED GALAXIES
    # -----------------
    
    mask_lum = entry["Selections"]["Isolated"]["Luminous"]
    mask_dark = entry["Selections"]["Isolated"]["Dark"]
    mask_all = np.logical_or(mask_lum, mask_dark)
    
    # Plot luminous isolated:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[1].plot(vmax[mask_lum], cnt_lum,
                 c=color[i][0], linestyle='dashed')
    
    
    # Plot all isolated:
    cnt_all = np.arange(1, np.sum(mask_all) + 1)[::-1]
    axes[1].plot(vmax[mask_all], cnt_all,
                 c=color[i][0], linestyle='solid')
  
    # Fill area of dark isolated:
    axes[1].fill(
        np.append(vmax[mask_lum], vmax[mask_all][::-1]),
        np.append(cnt_lum, cnt_all[::-1]),
        color=color[i][2],
        alpha=0.3
    )

            
# Make dummy plots for the legend:   
l = []
for c in color:
    l_dum, = axes[1].plot([], [], c=c[0], linestyle="solid")
    l.append(l_dum)
sim_legend = axes[1].legend(l, list(data.keys()), loc="upper right")
axes[1].add_artist(sim_legend)


axes[0].plot([], [], c=color[0][0], linestyle="solid", label="All")
axes[0].plot([], [], c=color[0][0], linestyle="dashed", label="Luminous")
axes[0].legend(loc="upper right")
    
plt.tight_layout()
plt.savefig(filename, dpi=200)